<font color='red'> Penalty kick application based on 'Professionals Play Minimax' by Ignacio Palacios-Huerta (2003). </font>

### Imports

In [1]:
from bokeh.io import show, output_notebook, save, output_file
from bokeh.models import (Plot, TapTool, ColumnDataSource, LabelSet, 
                          StaticLayoutProvider, Circle, MultiLine)
from bokeh.models.widgets import Div
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.plotting import figure
from bokeh.models.renderers import GraphRenderer, GlyphRenderer
from bokeh.layouts import layout, row, column, gridplot
from bokeh.models import (CustomJSHover, CustomJS, Button, Slider, DataTable, 
                          ColumnDataSource, TableColumn, Rect, Dropdown, 
                          Legend, LegendItem, Circle, HoverTool)
from bokeh.models.glyphs import Text
from bokeh.events import ButtonClick


In [2]:
output_notebook()

Loading BokehJS ...

### Define Figure and Sprites

In [3]:
game_figure = figure(tools = "", toolbar_location = None, 
                     title = 'FIFA 2020 Penalty Simulator',
                     plot_width = 600, plot_height = 480, 
                     x_range = (0, 100), y_range = (0, 90))
game_figure.title.text_font_size = '15pt'

#Hide Axes and Gridlines
game_figure.xaxis.visible = False
game_figure.yaxis.visible = False
game_figure.xgrid.grid_line_color = None
game_figure.ygrid.grid_line_color = None
game_figure.outline_line_color = None

#Background Color
game_figure.background_fill_color = "green"

#Goal Posts and Lines
game_figure.multi_line([[24, 76, 78, 22, 24], 
                        [3,  12, 88, 97, 3 ], 
                        [34, 34, 66, 66]],
                       [[63, 63, 47, 47, 63], 
                        [15, 63, 63, 15, 15], 
                        [63, 82, 82, 63]],
                       color = ["lightgreen", 
                                "lightgreen", 
                                "whitesmoke"],
                       alpha = [1, 1, 1], 
                       line_width = 4)

#Striker Box
game_figure.quadratic(33, 15, 67, 15, 50, 2, 
                      color = 'lightgreen', 
                      line_width = 4)

#Goalie Sprite 
goalie_head = Circle(x = 50, y = 69, fill_color = 'red', 
                     line_width = 2, size = 17)
goalie_body = Rect(x = 50, y = 65, width = 3, height = 4, 
                   angle = 0, fill_color = 'red', line_width = 2)
game_figure.add_glyph(goalie_head)
game_figure.add_glyph(goalie_body)

#Ball 
ball = Circle(x = 50, y = 13, fill_color = 'whitesmoke', 
              line_width = 2, size = 17)
game_figure.add_glyph(ball);

#Striker Sprite 
game_figure.add_glyph(Circle(x = 50, y = 16, fill_color = 'lightblue', 
                             line_width = 2, size = 21)) # head
game_figure.add_glyph(Rect(x = 50, y = 11, width = 4, height = 6, 
                           angle = 0, fill_color = 'lightblue', line_width = 2)); #body

In [4]:
#Variables to Configure Section
SCORED_COLOR = "#3F6750"
BLOCKEDL_COLOR = "#64A580"
BLOCKEDM_COLOR = "#8AD3AA"
BLOCKEDR_COLOR = "#CBEBD9"

FIGURE1_WIDTH = 300
FIGURE1_HEIGHT = 240
LEGEND1_WIDTH = 191
LEGEND2_WIDTH = 267

LL_LOC = 0.5
LM_LOC = 1.5
LR_LOC = 2.5
RL_LOC = 3.5
RM_LOC = 4.5
RR_LOC = 5.5

BAR_WIDTH = 0.9

#Create Figure
game_stats_figure_1 = figure(tools = "", 
                             toolbar_location = None, 
                             title = 'Shot Status Statistics',
                             plot_width = FIGURE1_WIDTH, 
                             plot_height = FIGURE1_HEIGHT, 
                             x_range = (0, 6), y_range = (0, 50), 
                             visible = False)
  
game_stats_figure_1.title.text_font_size = '8pt'

#Configure Gridlines And Axes
game_stats_figure_1.xaxis.visible = False
game_stats_figure_1.yaxis.visible = True
game_stats_figure_1.xgrid.grid_line_color = None
game_stats_figure_1.ygrid.grid_line_color = None
game_stats_figure_1.outline_line_color = None

#Set Figure Background Color
game_stats_figure_1.background_fill_color = "white"

#Create Data Bars:
#Create Scored Bars:
ll_scored_bar = Rect(x = LL_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                     fill_color = SCORED_COLOR, line_width = 0)
lm_scored_bar = Rect(x = LM_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                     fill_color = SCORED_COLOR, line_width = 0)
lr_scored_bar = Rect(x = LR_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                     fill_color = SCORED_COLOR, line_width = 0)
rl_scored_bar = Rect(x = RL_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                     fill_color = SCORED_COLOR, line_width = 0)
rm_scored_bar = Rect(x = RM_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                     fill_color = SCORED_COLOR, line_width = 0)
rr_scored_bar = Rect(x = RR_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                     fill_color = SCORED_COLOR, line_width = 0)

game_stats_figure_1.add_glyph(ll_scored_bar)
game_stats_figure_1.add_glyph(lm_scored_bar)
game_stats_figure_1.add_glyph(lr_scored_bar)
game_stats_figure_1.add_glyph(rl_scored_bar)
game_stats_figure_1.add_glyph(rm_scored_bar)
game_stats_figure_1.add_glyph(rr_scored_bar)

#Create Blocked By Goalie Going Left Bars:
ll_blocked_left_bar = Rect(x = LL_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                           fill_color = BLOCKEDL_COLOR, line_width = 0)
lm_blocked_left_bar = Rect(x = LM_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                           fill_color = BLOCKEDL_COLOR, line_width = 0)
lr_blocked_left_bar = Rect(x = LR_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                           fill_color = BLOCKEDL_COLOR, line_width = 0)
rl_blocked_left_bar = Rect(x = RL_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                           fill_color = BLOCKEDL_COLOR, line_width = 0)
rm_blocked_left_bar = Rect(x = RM_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                           fill_color = BLOCKEDL_COLOR, line_width = 0)
rr_blocked_left_bar = Rect(x = RR_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                           fill_color = BLOCKEDL_COLOR, line_width = 0)

game_stats_figure_1.add_glyph(ll_blocked_left_bar)
game_stats_figure_1.add_glyph(lm_blocked_left_bar)
game_stats_figure_1.add_glyph(lr_blocked_left_bar)
game_stats_figure_1.add_glyph(rl_blocked_left_bar)
game_stats_figure_1.add_glyph(rm_blocked_left_bar)
game_stats_figure_1.add_glyph(rr_blocked_left_bar)

#Create Blocked By Goalie Going Middle Bars:
ll_blocked_middle_bar = Rect(x = LL_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                             fill_color = BLOCKEDM_COLOR, line_width = 0)
lm_blocked_middle_bar = Rect(x = LM_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                             fill_color = BLOCKEDM_COLOR, line_width = 0)
lr_blocked_middle_bar = Rect(x = LR_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                             fill_color = BLOCKEDM_COLOR, line_width = 0)
rl_blocked_middle_bar = Rect(x = RL_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                             fill_color = BLOCKEDM_COLOR, line_width = 0)
rm_blocked_middle_bar = Rect(x = RM_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                             fill_color = BLOCKEDM_COLOR, line_width = 0)
rr_blocked_middle_bar = Rect(x = RR_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                             fill_color = BLOCKEDM_COLOR, line_width = 0)

game_stats_figure_1.add_glyph(ll_blocked_middle_bar)
game_stats_figure_1.add_glyph(lm_blocked_middle_bar)
game_stats_figure_1.add_glyph(lr_blocked_middle_bar)
game_stats_figure_1.add_glyph(rl_blocked_middle_bar)
game_stats_figure_1.add_glyph(rm_blocked_middle_bar)
game_stats_figure_1.add_glyph(rr_blocked_middle_bar)

#Create Blocked By Goalie Going Right Bars:
ll_blocked_right_bar = Rect(x = LL_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                            fill_color = BLOCKEDR_COLOR, line_width = 0)
lm_blocked_right_bar = Rect(x = LM_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                            fill_color = BLOCKEDR_COLOR, line_width = 0)
lr_blocked_right_bar = Rect(x = LR_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                            fill_color = BLOCKEDR_COLOR, line_width = 0)
rl_blocked_right_bar = Rect(x = RL_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                            fill_color = BLOCKEDR_COLOR, line_width = 0)
rm_blocked_right_bar = Rect(x = RM_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                            fill_color = BLOCKEDR_COLOR, line_width = 0)
rr_blocked_right_bar = Rect(x = RR_LOC, y = 0, width = BAR_WIDTH, height = 0, angle = 0, 
                            fill_color = BLOCKEDR_COLOR, line_width = 0)

game_stats_figure_1.add_glyph(ll_blocked_right_bar)
game_stats_figure_1.add_glyph(lm_blocked_right_bar)
game_stats_figure_1.add_glyph(lr_blocked_right_bar)
game_stats_figure_1.add_glyph(rl_blocked_right_bar)
game_stats_figure_1.add_glyph(rm_blocked_right_bar)
game_stats_figure_1.add_glyph(rr_blocked_right_bar)


#Creating The Legend:
#Add shapes to represent colors (Cannot be seen as radius=0):
l1scored = game_stats_figure_1.circle(x = 0, y = 0, radius = 0, color = SCORED_COLOR)
l1blockedl = game_stats_figure_1.circle(x = 0, y = 0, radius = 0, color = BLOCKEDL_COLOR)
l2blockedm = game_stats_figure_1.circle(x = 0, y = 0, radius = 0, color = BLOCKEDM_COLOR)
l2blockedr = game_stats_figure_1.circle(x = 0, y = 0, radius = 0, color = BLOCKEDR_COLOR)

#Create Legends, Configure Placement And Names:
legend1 = Legend(items = [("Scored", [l1scored]), ("Goalie Blocked By Going Left", [l1blockedl])],
                 orientation = "horizontal", location = (int((FIGURE1_WIDTH - LEGEND1_WIDTH) / 2), 22),
                 label_text_font_size = "5pt", label_height = 10, padding = 0, border_line_alpha = 0,
                 background_fill_alpha = 0)
legend2 = Legend(items = [("Goalie Blocked By Going Middle", [l2blockedm]), 
                          ("Goalie Blocked By Going Right", [l2blockedr])],
                 orientation = "horizontal", location = (int((FIGURE1_WIDTH - LEGEND2_WIDTH) / 2), 50),
                 label_text_font_size = "5pt", label_height = 10, padding = 0, border_line_alpha = 0,
                 background_fill_alpha = 0)

game_stats_figure_1.add_layout(legend1, "below")
game_stats_figure_1.add_layout(legend2, "below")


#Creating A Data Source To Help With Hovering:
game_stats_figure_1_source_xs = [0.00, 0.50, 1.00, 
                                 1.00, 1.50, 2.00, 
                                 2.00, 2.50, 3.00, 
                                 3.00, 3.50, 4.00, 
                                 4.00, 4.50, 5.00, 
                                 5.00, 5.50, 6.00]

game_stats_figure_1_source_scored_ys = [0.00, 0.00, 0.00, 
                                        0.00, 0.00, 0.00, 
                                        0.00, 0.00, 0.00, 
                                        0.00, 0.00, 0.00, 
                                        0.00, 0.00, 0.00, 
                                        0.00, 0.00, 0.00]

game_stats_figure_1_source_blockedl_ys = [0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00]

game_stats_figure_1_source_blockedm_ys = [0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00]

game_stats_figure_1_source_blockedr_ys = [0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00, 
                                          0.00, 0.00, 0.00]

game_stats_figure_1_source = ColumnDataSource(data = dict(x = game_stats_figure_1_source_xs,
                                                          scored_y = game_stats_figure_1_source_scored_ys,
                                                          blockedl_y = game_stats_figure_1_source_blockedl_ys,
                                                          blockedm_y = game_stats_figure_1_source_blockedm_ys,
                                                          blockedr_y = game_stats_figure_1_source_blockedr_ys,
                                                          hoverxy = [0,0,0,0,0,0,0,0,0,
                                                                     0,0,0,0,0,0,0,0,0], 
                                                          hoverhovering = [0,0,0,0,0,0,0,0,0,
                                                                           0,0,0,0,0,0,0,0,0],
                                                          hovershow = [0,1,0,0,1,0,0,1,0,
                                                                       0,1,0,0,1,0,0,1,0]))


#Creating Invisible Line Glyphs to Help With Hovering
game_stats_figure_1.line('x', 'scored_y', source = game_stats_figure_1_source, line_width = 1,
                         line_alpha = 0, line_color = SCORED_COLOR, name = 'scored_y')
game_stats_figure_1.line('x', 'blockedl_y', source = game_stats_figure_1_source, line_width = 1, 
                         line_alpha = 0, line_color = BLOCKEDL_COLOR, name = 'blockedl_y')
game_stats_figure_1.line('x', 'blockedm_y', source = game_stats_figure_1_source, line_width = 1, 
                         line_alpha = 0, line_color = BLOCKEDM_COLOR, name = 'blockedm_y')
game_stats_figure_1.line('x', 'blockedr_y', source = game_stats_figure_1_source, line_width = 1, 
                         line_alpha = 0, line_color = BLOCKEDR_COLOR, name = 'blockedr_y')


#Creating a Custom HoverTool:
#Code for the CustomJSHovers for the HoverTool:
custom_hover_coordinates_code = """
var x = special_vars.x; 
var y = special_vars.y;

var modified_x;
var modified_y;
var index = 0;

const data = game_stats_figure_1_source.data;

for(var i = 6; i >= 1; i--){
    if(x < i){
        modified_x = (i - 0.5);
        index = (3*i - 2);
    }
}

var closest_y;

var scored_val = data['scored_y'][index];
var blockedl_val = data['blockedl_y'][index];
var blockedm_val = data['blockedm_y'][index];
var blockedr_val = data['blockedr_y'][index];

closest_y = scored_val;

if((y > closest_y 
    && blockedl_val != scored_val)){
    closest_y = blockedl_val;
}
if((y > closest_y
    && blockedm_val != blockedl_val)){
    closest_y = blockedm_val;
}
if((y > closest_y 
    && blockedr_val != blockedm_val)){
    closest_y = blockedr_val;
}


modified_y = closest_y;

return("(" + modified_x.toString() + "," 
       + modified_y.toString() + ")" );
"""

custom_hover_hovering_code = """
var x = special_vars.x; 
var y = special_vars.y;

var modified_x;
var modified_y;
var index = 0;

const data = game_stats_figure_1_source.data;

for(var i = 6; i >= 1; i--){
    if(x < i){
        modified_x = (i - 0.5);
        index = (3*i - 2);
    }
}

var closest_y;

var scored_val = data['scored_y'][index];
var blockedl_val = data['blockedl_y'][index];
var blockedm_val = data['blockedm_y'][index];
var blockedr_val = data['blockedr_y'][index];

closest_y = scored_val;
var hovering = "Scored";

if((y > closest_y 
    && blockedl_val != scored_val)){
    closest_y = blockedl_val;
    hovering = "Goalie Left";
}
if((y > closest_y
    && blockedm_val != blockedl_val)){
    closest_y = blockedm_val;
    hovering = "Goalie Middle";
}
if((y > closest_y 
    && blockedr_val != blockedm_val)){
    closest_y = blockedr_val;
    hovering = "Goalie Right";
}

return(hovering);
"""

custom_hover_code = """
var index = special_vars.index;
const data = game_stats_figure_1_source.data;

if(data['hovershow'][index] == 0){
    return " hidden ";
}

var y = special_vars.y;

var closest_y;

var scored_val = data['scored_y'][index];
var blockedl_val = data['blockedl_y'][index];
var blockedm_val = data['blockedm_y'][index];
var blockedr_val = data['blockedr_y'][index];

closest_y = scored_val;
var hovering = "scored_y";

if((y > closest_y 
    && blockedl_val != scored_val)){
    closest_y = blockedl_val;
    hovering = "blockedl_y";
}
if((y > closest_y
    && blockedm_val != blockedl_val)){
    closest_y = blockedm_val;
    hovering = "blockedm_y";
}
if((y > closest_y 
    && blockedr_val != blockedm_val)){
    closest_y = blockedr_val;
    hovering = "blockedr_y";
}

if(special_vars.name != hovering){
    return " hidden ";
}

return " ";
"""

#Create CustomJSHovers for the HoverTool:
coordinates_custom = CustomJSHover(code = custom_hover_coordinates_code,
                                   args = dict(game_stats_figure_1_source = game_stats_figure_1_source))
hovering_custom = CustomJSHover(code = custom_hover_hovering_code,
                                args = dict(game_stats_figure_1_source = game_stats_figure_1_source))
custom_hover = CustomJSHover(code = custom_hover_code, 
                             args = dict(game_stats_figure_1_source = game_stats_figure_1_source))

#Creating and Configuring The Custom HoverTool:
fig_1_custom_tooltip = """

    <div @hovershow{custom}>
        <span style='font-size: 10px;'>Closest Data Coords:</span>
        <span style='font-size: 10px;'>@hoverxy{custom}</span>
    </div>
    <div @hovershow{custom}>
        <span style='font-size: 10px;'>Closest Data:</span>
        <span style='font-size: 10px;'>@hoverhovering{custom}</span>
    </div>
    <div @hovershow{custom}>
        <span style='font-size: 10px;'>Hovered (x,y):</span>
        <span style='font-size: 10px;'>($x,$y)</span>
    </div>

"""
game_stats_figure_1.add_tools(HoverTool(tooltips = fig_1_custom_tooltip,
                                        formatters = {'@hoverxy' : coordinates_custom,
                                                      '@hoverhovering' : hovering_custom,
                                                      '@hovershow' : custom_hover},
                                        mode = "vline"))

In [5]:
#Variables to Configure Section
FIGURE2_WIDTH = 300
FIGURE2_HEIGHT = 240
DOT_OUTLINE_COLOR = "#B56464"
DOT_COLOR = "#CE7D7D"
DOT_SIZE = 5
HIGHLIGHT_OUTLINE_COLOR = "#6464B5"
HIGHLIGHT_COLOR = "#7D7DCE"
HIGHLIGHT_SIZE = 10

#Create Figure
game_stats_figure_2 = figure(tools = "box_zoom, wheel_zoom, pan", 
                             toolbar_location = "below",
                             toolbar_sticky = True,
                             title = 'Score Over Iteration',
                             plot_width = FIGURE2_WIDTH, 
                             plot_height = FIGURE2_HEIGHT, 
                             x_range = (0, 50), y_range = (-50, 50), 
                             visible = False)
  
game_stats_figure_2.title.text_font_size = '8pt'

#Configure Gridlines And Axes
game_stats_figure_2.xaxis.visible = True
game_stats_figure_2.yaxis.visible = True
game_stats_figure_2.xgrid.grid_line_color = None
game_stats_figure_2.ygrid.grid_line_color = None
game_stats_figure_2.outline_line_color = None

#Set Figure Background Color
game_stats_figure_2.background_fill_color = "white"

#Create Data Source For Figure:
game_stats_figure_2_source_xs = []
game_stats_figure_2_source_ys = []
game_stats_figure_2_source_heights = []
game_stats_figure_2_source_highlight_alphas = []
#Fill the Lists
for i in range(51):
    game_stats_figure_2_source_xs.append(i)
    game_stats_figure_2_source_ys.append(0)
    game_stats_figure_2_source_heights.append(100)
    game_stats_figure_2_source_highlight_alphas.append(0)
    
game_stats_figure_2_source = ColumnDataSource(data = dict(xs = game_stats_figure_2_source_xs,
                                                          ys = game_stats_figure_2_source_ys,
                                                          heights = game_stats_figure_2_source_heights,
                                                          highlight_alphas = game_stats_figure_2_source_highlight_alphas))

#Plot Points for Figure:
game_stats_figure_2.circle_dot('xs', 'ys', source = game_stats_figure_2_source, size = DOT_SIZE,
                               color = DOT_OUTLINE_COLOR, fill_color = DOT_COLOR)

#Highlight Dots to Show Selection When Hovering:
game_stats_figure_2.circle_dot('xs', 'ys', source = game_stats_figure_2_source, size = HIGHLIGHT_SIZE,
                               color = HIGHLIGHT_OUTLINE_COLOR, fill_color = HIGHLIGHT_COLOR, alpha = 'highlight_alphas')


#Invisible Boxes to Aid In Hovering:
game_stats_figure_2.rect(x = 'xs', y = 0, source = game_stats_figure_2_source, 
                         width = 1, height = 'heights', fill_color = DOT_COLOR, alpha = 0)

#Create Hover Tool for Figure:
fig_2_xs_code = """
var index = special_vars.index;

for(var i = 0; 
    i < game_stats_figure_2_source.data['highlight_alphas'].length; 
    i++){
    game_stats_figure_2_source.data['highlight_alphas'][i] = 0;
}
game_stats_figure_2_source.data['highlight_alphas'][index] = 1;
game_stats_figure_2_source.change.emit();

return(game_stats_figure_2_source.data['xs'][index].toString())
"""

fig_2_ys_code = """
var index = special_vars.index;

return(game_stats_figure_2_source.data['ys'][index].toString())
"""

fig_2_xs_custom = CustomJSHover(code = fig_2_xs_code,
                                args = dict(game_stats_figure_2_source = game_stats_figure_2_source))
fig_2_ys_custom = CustomJSHover(code = fig_2_ys_code,
                                args = dict(game_stats_figure_2_source = game_stats_figure_2_source))

fig_2_custom_tooltip = """
<div>
    <span style='font-size: 10px;'>Iteration:</span>
    <span style='font-size: 10px;'>@xs{custom}</span>
</div>
<div @hovershow{custom}>
    <span style='font-size: 10px;'>Score:</span>
    <span style='font-size: 10px;'>@ys{custom}</span>
</div>
"""

game_stats_figure_2.add_tools(HoverTool(tooltips = fig_2_custom_tooltip,
                                        formatters = { '@xs' : fig_2_xs_custom,
                                                       '@ys' : fig_2_ys_custom},
                                        mode = "mouse",
                                        point_policy = "follow_mouse"))

### Define Labels

In [6]:
scr_text = ColumnDataSource({'x' : [2, 70, 2, 14, 14],
                             'y' : [86, 86, 5, 40, 32],
                             'text' : ['Rounds played: 0',
                                      'Total score: 0',
                                      '',
                                      '',
                                      '']})

labels = Text(x = "x", y = "y", text = 'text', text_color = "whitesmoke",
              text_font_size = '15pt', x_offset = 0, y_offset = +9,
              text_baseline = 'ideographic', text_align = 'left')

game_figure.add_glyph(scr_text, labels);

### Define State Variables as Divs

In [7]:
nround = Div(text = '0', visible = False) # total number of rounds completed
score = Div(text = '0', visible = False) # current score
freq_left_foot = Div(text = '0,0,0', visible = False) # frequency of left, middle, and right kicks (left-footed)
freq_right_foot = Div(text = '0,0,0', visible = False) # frequency of left, middle, and right kicks (right-footed)
kicker_foot = Div(text = '', visible = False) # current footedness of the kicker
kicker_kick = Div(text = '', visible = False) # current direction the kicker will kick

iterations_to_run = Div(text = '50', visible = False) # Amount of iterations for the game to run

strategy_to_use = Div(text = 'Not Set', visible = False)

LL_chance = 1/6
LM_chance = 1/6
LR_chance = 1/6
RL_chance = 1/6
RM_chance = 1/6
RR_chance = 1/6

ll_scored = Div(text = '0', visible = False)
lm_scored = Div(text = '0', visible = False)
lr_scored = Div(text = '0', visible = False)
rl_scored = Div(text = '0', visible = False)
rm_scored = Div(text = '0', visible = False)
rr_scored = Div(text = '0', visible = False)

ll_blocked_left = Div(text = '0', visible = False)
lm_blocked_left = Div(text = '0', visible = False)
lr_blocked_left = Div(text = '0', visible = False)
rl_blocked_left = Div(text = '0', visible = False)
rm_blocked_left = Div(text = '0', visible = False)
rr_blocked_left = Div(text = '0', visible = False)

ll_blocked_middle = Div(text = '0', visible = False)
lm_blocked_middle = Div(text = '0', visible = False)
lr_blocked_middle = Div(text = '0', visible = False)
rl_blocked_middle = Div(text = '0', visible = False)
rm_blocked_middle = Div(text = '0', visible = False)
rr_blocked_middle = Div(text = '0', visible = False)

ll_blocked_right = Div(text = '0', visible = False)
lm_blocked_right = Div(text = '0', visible = False)
lr_blocked_right = Div(text = '0', visible = False)
rl_blocked_right = Div(text = '0', visible = False)
rm_blocked_right = Div(text = '0', visible = False)
rr_blocked_right = Div(text = '0', visible = False)

### Define Buttons

In [8]:
#Start and Next Buttons:
b_start = Button(label = "Begin", button_type = "success", 
                 sizing_mode = 'scale_width', width_policy = 'fit')
b_next = Button(label = "Next round", button_type = "success", 
                sizing_mode = 'scale_width', width_policy = 'fit', disabled = True)

#Right and Left Footed Buttons:
b_fl = Button(label = "Left-Footed", button_type = "success", 
              sizing_mode = 'scale_width', width_policy = 'fit', disabled = True)
b_fr = Button(label = "Right-Footed", button_type = "success", 
              sizing_mode = 'scale_width', width_policy = 'fit', disabled = True)

#Right, Middle, and Left Kick Buttons:
bl = Button(label = "Left", button_type = "success", sizing_mode = 'scale_width', 
            width_policy = 'fit', disabled = False, visible = False)
bm = Button(label = "Middle", button_type = "success", sizing_mode = 'scale_width', 
            width_policy = 'fit', disabled = False, visible = False)
br = Button(label = "Right", button_type = "success", sizing_mode = 'scale_width', 
            width_policy = 'fit', disabled = False, visible = False)

#Shoot Button:
b_shoot = Button(label = "SHOOT!", button_type = "success", sizing_mode = 'scale_width', 
                 width_policy = 'fit', disabled = False, visible = False)

#Automate Buttons:
b_automate = Button(label = "Automate", button_type = "success", 
                    sizing_mode = 'scale_width', width_policy = 'fit')
b_start_automate = Button(label = "Start", button_type = "success", sizing_mode = 'scale_width', 
                          width_policy = 'fit', disabled = False, visible = False)

#Automate Next Round Button:
b_auto_next = Button(label = "Next", button_type = "success", sizing_mode = 'scale_width', 
                     width_policy = 'fit', disabled = False, visible = False)

### Define Sliders

In [9]:
#Aim Direction Chance Weight Sliders:
LL_aim_slider = Slider(start = 0, end = 1, value = 1/6, step = 0.01, 
                       title = "LL Aim Weight", disabled = False, visible = False)
LM_aim_slider = Slider(start = 0, end = 1, value = 1/6, step = 0.01, 
                       title = "LM Aim Weight", disabled = False, visible = False)
LR_aim_slider = Slider(start = 0, end = 1, value = 1/6, step = 0.01, 
                       title = "LR Aim Weight", disabled = False, visible = False)
RL_aim_slider = Slider(start = 0, end = 1, value = 1/6, step = 0.01, 
                       title = "RL Aim Weight", disabled = False, visible = False)
RM_aim_slider = Slider(start = 0, end = 1, value = 1/6, step = 0.01, 
                       title = "RM Aim Weight", disabled = False, visible = False)
RR_aim_slider = Slider(start = 0, end = 1, value = 1/6, step = 0.01, 
                       title = "RR Aim Weight", disabled = False, visible = False)

#Iterations To Run Slider:
iterations_slider = Slider(start = 10, end = 500, value = 50, step = 1, 
                           title = "Iterations To Run", disabled = False, visible = False)

### Define Dropdown Menu

In [10]:
#CPU Strategy to Use Dropdown:
menu = [("Fictitious_Play", "Fictitious_Play"), 
        ("Mixed_Strategy", "Mixed_Strategy")]
strategy_dropdown = Dropdown(label = "CPU strategy to Use", menu = menu, 
                             button_type = "warning", disabled = False, visible = False)

### Define Automation Distribution Tracking Table

In [11]:
#Make Automation Distribution Tracking Table:
distribution_data = dict(footedness = ["Left", "Left", "Left", 
                                       "Right", "Right", "Right"], 
                         aim_direction = ["Left", "Middle", "Right", 
                                          "Left", "Middle", "Right"],
                         freq = [0, 0, 0, 0, 0, 0], 
                         decisions = [0, 0, 0, 0, 0, 0], 
                         goalie_perceived_risks = [0, 0, 0, 0, 0, 0],
                         striker_score_chance = [0, 0, 0, 0, 0, 0], 
                         striker_score_roll = [0, 0, 0, 0, 0, 0])

automation_distribution_table_source = ColumnDataSource(distribution_data)

distribution_columns = [TableColumn(field = "footedness", title = "Striker Footedness", width = 101),
                        TableColumn(field = "aim_direction", title = "Striker Aim Direction", width = 107),
                        TableColumn(field = "freq", title = "Frequency", width = 60),
                        TableColumn(field = "decisions", title = "Goalie Decisions", width = 90),
                        TableColumn(field = "goalie_perceived_risks", title = "Goalie's Perceived Risks", width = 130),
                        TableColumn(field = "striker_score_chance", title = "Striker's Score Chance", width = 120),
                        TableColumn(field = "striker_score_roll", title = "Striker's Score Roll", width = 103)]

automation_distribution_table = DataTable(source = automation_distribution_table_source, 
                                          columns = distribution_columns, width = 711, height = 280, 
                                          autosize_mode = "force_fit", 
                                          sizing_mode = "scale_width", visible = False, 
                                          fit_columns = False)

### Define Callbacks

In [12]:
#Next Button:
b_next_code = """
b_shoot.visible = false;

b_fr.visible = true;
b_fl.visible = true;
b_fr.disabled = false;
b_fl.disabled = false;

bl.visible = false;
bm.visible = false;
br.visible = false;

b_next.disabled = true;

txt.data['text'][2] = '';
txt.data['text'][3] = 'Choose a right- or left footed kicker!';
txt.change.emit();

ball.x = 50;
ball.y = 13;
goalie_head.x = 50;
goalie_body.x = 50;
"""

b_next_click = CustomJS(args = dict(b_start = b_start, b_fl = b_fl, b_fr = b_fr, bl = bl, bm = bm, br = br,
                                    txt = scr_text, b_shoot = b_shoot, b_next = b_next, ball = ball,
                                    goalie_head = goalie_head, goalie_body = goalie_body),
                        code = b_next_code)

b_next.js_on_click(b_next_click)

In [13]:
#Start Button:
b_start_code = b_next_code + """
if (b_start.label == 'Begin'){
    b_start.label = 'Restart';
}
else{
    txt.data['text'][0] = 'Rounds played: 0';
    txt.data['text'][1] = 'Total score: 0';
    txt.data['text'][2] = '';
    txt.data['text'][3] = '';
    txt.data['text'][4] = '';
    nround.text = '0';
    score.text = '0';
    freq_left_foot.text = '0,0,0';
    freq_right_foot.text = '0,0,0';
    kicker_foot.text = '';
    kicker_kick.text = '';
    txt.change.emit();
    b_start.label = 'Begin';
}
"""

b_start_click = CustomJS(args = dict(b_start = b_start, b_fl = b_fl, b_fr = b_fr, bl = bl, bm = bm, br = br,
                                     txt = scr_text, b_shoot=b_shoot, b_next = b_next, ball = ball,
                                     goalie_head = goalie_head, goalie_body = goalie_body, nround=nround,
                                     score = score, freq_left_foot = freq_left_foot, 
                                     freq_right_foot = freq_right_foot, kicker_foot = kicker_foot, 
                                     kicker_kick = kicker_kick),
                         code = b_start_code)

b_start.js_on_click(b_start_click)

In [14]:
#Create the Striker Foot Buttons:

#Select the Foot Type of the Striker Buttons:
def b_f_click(foot):
    code = """
    b_fl.disabled = true;
    b_fr.disabled = true;
    b_fl.visible = false;
    b_fr.visible = false;
    bl.visible = true;
    bm.visible = true;
    br.visible = true;
    txt.data['text'][3] = 'Choose where to kick!';
    txt.data['text'][2] = '""" + foot + """-footed kicker.';
    kicker_foot.text = '""" + foot + """';
    txt.change.emit();
    """
    return CustomJS(args = dict(b_fl = b_fl, b_fr = b_fr, bl = bl, bm = bm, br = br,
                                txt = scr_text, kicker_foot = kicker_foot),
                    code = code)

b_fl.js_on_click(b_f_click('Left'))
b_fr.js_on_click(b_f_click('Right'))

In [15]:
#Create the Kicker Kick Buttons:

#Select the Kick Direction of the Kicker Kick Buttons:
def b_kick_click(kick):
    code = """
    b_shoot.visible = true;
    bl.visible = false;
    bm.visible = false;
    br.visible = false;
    txt.data['text'][3] = '';
    var current = txt.data['text'][2];
    txt.data['text'][2] = current + ' Kicking """+kick+""".';
    kicker_kick.text= '"""+kick+"""';
    txt.change.emit();
    b_shoot.disabled = false;
    """
    return CustomJS(args = dict(b_shoot = b_shoot, bl = bl, bm = bm, br = br,
                                txt = scr_text, kicker_kick = kicker_kick),
                    code = code)

bl.js_on_click(b_kick_click('Left'))
bm.js_on_click(b_kick_click('Middle'))
br.js_on_click(b_kick_click('Right'))

In [16]:
# shoot button
b_shoot_code = """
// Define probability matrix
var p = {'Right' : {'LeftLeft' : 0.55,
                    'LeftMiddle' : 0.65,
                    'LeftRight' : 0.93,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.95,
                    'RightMiddle' : 0.73,
                    'RightRight' : 0.70},
         'Left' :  {'LeftLeft' : 0.67,
                    'LeftMiddle' : 0.70,
                    'LeftRight' : 0.96,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.87,
                    'RightMiddle' : 0.65,
                    'RightRight' : 0.61}};

// Choose best action
var freq;

if (kicker_foot.text == 'Right'){
    freq = freq_right_foot.text.split(',').map(Number);
}
else{
    freq = freq_left_foot.text.split(',').map(Number);
}

var kicker_action = 'Left';
var expected = (freq[0] * p[kicker_foot.text]['LeftLeft']
                + freq[1]*p[kicker_foot.text]['MiddleLeft']
                + freq[2]*p[kicker_foot.text]['RightLeft']);
var actions = ["Left", "Middle", "Right"];

for (var i = 0; i < 3; i++){
    var val = (freq[0] * p[kicker_foot.text]['Left'+actions[i]]
               + freq[1]*p[kicker_foot.text]['Middle'+actions[i]]
               + freq[2]*p[kicker_foot.text]['Right'+actions[i]]);
    if (val < expected){
        kicker_action = actions[i];
        expected = val;
    }
}

// Determine if goal
var goal = 1;
if (Math.random() > p[kicker_foot.text][kicker_kick.text + kicker_action]){
    goal = -1;
}

// Animate ball and goalie
ball.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_kick.text];
ball.y = 63;
goalie_head.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_action];
goalie_body.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_action];

// Add to frequency history
var dict = {'Left' : 0,
            'Middle' : 1,
            'Right' : 2};
if (kicker_foot.text == 'Right'){
    var freq = freq_right_foot.text.split(',');
    freq[dict[kicker_kick.text]] = parseInt(freq[dict[kicker_kick.text]]) + 1;
    freq_right_foot.text = freq.toString();
}
else{
    var freq = freq_left_foot.text.split(',');
    freq[dict[kicker_kick.text]] = parseInt(freq[dict[kicker_kick.text]]) + 1;
    freq_left_foot.text = freq.toString();
}

// Update text
var n = (parseInt(nround.text) + 1);
nround.text = n.toString();
txt.data['text'][0] = 'Rounds played: ' + n;

if (goal == 1){
    txt.data['text'][3] = 'GOAL!';
}
else{
    txt.data['text'][3] = 'Blocked';
}

var s = parseInt(score.text) + goal;
score.text = s.toString();
txt.data['text'][1] = 'Total score: ' + s;

txt.change.emit();

// Allow starting next round
b_shoot.disabled = true;
b_next.disabled = false;
"""

b_shoot_click = CustomJS(args = dict(kicker_foot = kicker_foot, freq_right_foot = freq_right_foot,
                                     freq_left_foot = freq_left_foot, kicker_kick = kicker_kick,
                                     ball = ball, goalie_head = goalie_head, goalie_body = goalie_body,
                                     nround = nround, txt = scr_text, score = score, b_shoot = b_shoot,
                                     b_next = b_next),
                         code = b_shoot_code)

b_shoot.js_on_click(b_shoot_click)

In [17]:
#Make Automation Decision Table:
data = dict(footedness = ["Left", "Left", "Left", 
                          "Right", "Right", "Right"], 
            aim_direction = ["Left", "Middle", "Right", 
                             "Left", "Middle", "Right"],
            chances = [LL_chance, LM_chance, LR_chance, 
                       RL_chance, RM_chance, RR_chance])

automation_table_source = ColumnDataSource(data)

columns = [TableColumn(field = "footedness", title = "Striker Footedness"),
           TableColumn(field = "aim_direction", title = "Striker Aim Direction"),
           TableColumn(field = "chances", title = "Chance")]

automation_table = DataTable(source = automation_table_source, columns = columns, 
                             width = 600, height = 280, autosize_mode = "force_fit",  
                             visible = False)

In [18]:
# automate button
b_automate_code = """
b_automate.visible = false;

b_start.visible = false;
b_fl.visible = false;
b_fr.visible = false;
b_next.visible = false;

LL_aim_slider.visible = true;
LM_aim_slider.visible = true;
LR_aim_slider.visible = true;
RL_aim_slider.visible = true;
RM_aim_slider.visible = true;
RR_aim_slider.visible = true;

iterations_slider.visible = true;

strategy_dropdown.visible = true;

automation_table.visible = true;

txt.data['text'][2] = '';
txt.data['text'][3] = 'Choose a right- or left footed kicker!';
txt.change.emit();
"""

b_automate_click = CustomJS(args = dict(b_automate = b_automate, b_start = b_start, b_fl = b_fl, b_fr = b_fr, 
                                        b_next = b_next, LL_aim_slider = LL_aim_slider, LM_aim_slider = LM_aim_slider,
                                        LR_aim_slider = LR_aim_slider, RL_aim_slider = RL_aim_slider,
                                        RM_aim_slider = RM_aim_slider, RR_aim_slider = RR_aim_slider,
                                        iterations_slider = iterations_slider, strategy_dropdown = strategy_dropdown,
                                        automation_table = automation_table, txt = scr_text), 
                            code = b_automate_code)

b_automate.js_on_click(b_automate_click)

In [19]:
automate_start_code_initial_gui_display = """
b_start_automate.visible = false;
b_auto_next.visible = true;

LL_aim_slider.visible = false;
LM_aim_slider.visible = false;
LR_aim_slider.visible = false;
RL_aim_slider.visible = false;
RM_aim_slider.visible = false;
RR_aim_slider.visible = false;

iterations_slider.visible = false;

strategy_dropdown.visible = false;

automation_table.visible = false;
automation_distribution_table.visible = true;
"""

automate_loop_iteration_var_instantiations = """
// Define probability matrix
var p = {'Right' : {'LeftLeft' : 0.55,
                    'LeftMiddle' : 0.65,
                    'LeftRight' : 0.93,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.95,
                    'RightMiddle' : 0.73,
                    'RightRight' : 0.70},
         'Left' :  {'LeftLeft' : 0.67,
                    'LeftMiddle' : 0.70,
                   'LeftRight' : 0.96,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.87,
                    'RightMiddle' : 0.65,
                    'RightRight' : 0.61}};

//Obtain the Shot Aim chances from the column data source
    var chances_data = ChancesColumnDataSource.data;
    var chances = chances_data['chances'];
    var LL_chance = chances[0];
    var LM_chance = LL_chance + chances[1];
    var LR_chance = LM_chance + chances[2];
    var RL_chance = LR_chance + chances[3];
    var RM_chance = RL_chance + chances[4];

//Goalie Logic Decision Making Variables
    var chance_left = 1/3;
    var chance_middle = 1/3;
    var chance_right = 1/3;

    var danger_goalie_left = 0;
    var danger_goalie_middle = 0;
    var danger_goalie_right = 0;
    var selected_dict = 0;

    var total_sample_rolls;

    var goalie_action = "None";

//Loop Randomness Variable
    var roll = 0;

//Goalie Fictitious Learning Variables
    var selected_index = 0;

    var dist_data = DistributionColumnDataSource.data;
    var freq = dist_data['freq'];
    var decisions = dist_data['decisions'];

//Data Display Variables
    var perceived_risks = dist_data['goalie_perceived_risks'];
    var scoring_chance = dist_data['striker_score_chance'];
    var scoring_roll = dist_data['striker_score_roll'];

//Striker Kick Choice Variables
    var kicker_foot = 'none';
    var kicker_kick = 'none';

//Scoring Variables
    var score_chance = 0;
    var goal = 1;

//Animation Variables
    var animation_positions = {'Left' : [37, 43],
                               'Middle' : [47,53],
                               'Right' : [57, 63]};
    var bally = 63;
"""

automate_loop_iteration_display = """
txt.data['text'][0] = 'Rounds played: ' + rounds_played;

if (goal == 1){
    txt.data['text'][3] = 'GOAL!';
} 
else{
    txt.data['text'][3] = 'Blocked';
}

txt.data['text'][1] = 'Total score: ' + game_score;

txt.change.emit();

DistributionColumnDataSource.change.emit();
"""

automate_loop_roll_kicker_action = """
//Handle Automated Selection
roll = Math.random();    
    
if(roll <= LL_chance){
    kicker_foot = 'Left';
    kicker_kick = 'Left';
}
else if(roll <= LM_chance){
    kicker_foot = 'Left';
    kicker_kick = 'Middle';
}
else if(roll <= LR_chance){
    kicker_foot = 'Left';
    kicker_kick = 'Right';
}
else if(roll <= RL_chance){
    kicker_foot = 'Right';
    kicker_kick = 'Left';
}
else if(roll <= RM_chance){
    kicker_foot = 'Right';
    kicker_kick = 'Middle';
}
else{
    kicker_foot = 'Right';
    kicker_kick = 'Right';
}
"""

run_fictitious_play = """
//Handle Goalie Decision

if(kicker_foot == 'Left'){
    total_sample_rolls = freq[0] + freq[1] + freq[2];
    selected_dict = p['Left'];
    
    if(total_sample_rolls == 0){
        chance_left = 1/3;
        chance_middle = 1/3;
        chance_right = 1/3;
    }
    else{
        chance_left = freq[0] / total_sample_rolls;
        chance_middle = freq[1] / total_sample_rolls;
        chance_right = freq[2] / total_sample_rolls;
    }
}
else{
    total_sample_rolls = freq[3] + freq[4] + freq[5];
    selected_dict = p['Right'];
    if(total_sample_rolls == 0){
        chance_left = 1/3;
        chance_middle = 1/3;
        chance_right = 1/3;
    }
    else{
        chance_left = freq[3] / total_sample_rolls;
        chance_middle = freq[4] / total_sample_rolls;
        chance_right = freq[5] / total_sample_rolls;
    }
}

danger_goalie_left = (chance_left * selected_dict['LeftLeft']
                      + chance_middle*selected_dict['MiddleLeft']
                      + chance_right*selected_dict['RightLeft']);
danger_goalie_middle = (chance_left * selected_dict['LeftMiddle']
                        + chance_middle*selected_dict['MiddleMiddle']
                        + chance_right*selected_dict['RightMiddle']);
danger_goalie_right = (chance_left * selected_dict['LeftRight']
                       + chance_middle*selected_dict['MiddleRight']
                       + chance_right*selected_dict['RightRight']);
    

if(danger_goalie_left < danger_goalie_middle){
    if(danger_goalie_left < danger_goalie_right){
        goalie_action = "Left";
    }
    else if(danger_goalie_left == danger_goalie_right){
        roll = Math.random();
        if(roll <= 0.5){
            goalie_action = "Left";
        }
        else{
            goalie_action = "Right";
        }
    }
    else{
        goalie_action = "Right";
    }
}
else if (danger_goalie_left == danger_goalie_middle){
    roll = Math.random();
    if(roll <= 0.5){
        goalie_action = "Left";
    }
    else{
        goalie_action = "Middle";
    }
}
else{
    if(danger_goalie_middle < danger_goalie_right){
        goalie_action = "Middle";
    }
    else if(danger_goalie_middle == danger_goalie_right){
        roll = Math.random();
        if(roll <= 0.5){
            goalie_action = "Middle";
        }
        else{
            goalie_action = "Right";
        }
    }
    else{
        goalie_action = "Right";
    }
}
"""

run_optimal_mixed_strategy = """
roll = Math.random();
if(kicker_foot == 'Left'){
    if(roll <= 0.8){
        goalie_action = "Middle";
    }
    else{
        goalie_action = "Left";
    }
}
else if(kicker_foot == 'Right'){
    if(roll <= 0.7419){
        goalie_action = "Middle";
    }
    else{
        goalie_action = "Right";
    }

}
"""

automate_loop_handle_goalie_decision = """
if(strategy_to_use.text == "Fictitious_Play"){
    """ + run_fictitious_play + """
}
else{
    """ + run_optimal_mixed_strategy + """
}
"""

automate_loop_handle_scoring = """
//Handle Score Chance:

roll = Math.random();
score_chance = p[kicker_foot][kicker_kick+goalie_action];
if(roll <= score_chance){
    goal = 1;
}
else{
    goal = -1;
}
//Display Score Chance:

for(var i=0; i<=5; i++){
    scoring_chance[i] = 0;
    scoring_roll[i] = 0;
}

selected_index = 0;
if(kicker_foot == 'Right'){
    selected_index += 3;
}
if(kicker_kick == 'Middle'){
    selected_index += 1;
}
else if(kicker_kick == 'Right'){
    selected_index += 2;
}

scoring_chance[selected_index] = score_chance;
scoring_roll[selected_index] = roll.toString().substring(0, 8);

// Update text
    
var rounds_played = (parseInt(nround.text) + 1);
if(rounds_played >= parseInt(iterations_to_run.text)){
    b_auto_next.visible = false;
    game_figure.visible = false;
    automation_distribution_table.visible = false;
    game_stats_figure_1.visible = true;
    game_stats_figure_2.visible = true;
}
nround.text = rounds_played.toString();
    
var game_score = parseInt(score.text) + goal;
score.text = game_score.toString();
"""

automate_loop_animation = """
//Animate Scenario:

var animation_roll = 0;
var animation_slot = 0;
animation_roll = Math.random()

if(animation_roll <= 0.5){
    animation_slot = 1;
}

ball.x = animation_positions[kicker_kick][animation_slot];
ball.y = bally;

if(goal == -1){
    if(goalie_action == kicker_kick){
        goalie_head.x = ball.x;
        goalie_body.x = ball.x;
    }
    else{
        if(kicker_kick == "Right"){
            ball.x = 70;
        }
        else if(kicker_kick == "Left"){
            ball.x = 30;
        }
        else{
            ball.x = [30,70][animation_slot];
        }
    }

}
else{
    if(goalie_action == kicker_kick){
        if(animation_slot == 1){
            goalie_head.x = animation_positions[goalie_action][0];
            goalie_body.x = animation_positions[goalie_action][0];
        }
        else{
            goalie_head.x = animation_positions[goalie_action][1];
            goalie_body.x = animation_positions[goalie_action][1];
        }
    }
    else{
        if(animation_roll <= 0.5){
            animation_slot = 1;
        }
        else{
            animation_slot = 0;
        }
        goalie_head.x = animation_positions[goalie_action][animation_slot];
        goalie_body.x = animation_positions[goalie_action][animation_slot];
    }
}
"""

automate_loop_update_fictitious_decision_tracking = """
//Update Goalie Frequency Tracking:

selected_index = 0;
if(kicker_foot == 'Right'){
    selected_index += 3;
}
if(kicker_kick == 'Middle'){
    selected_index += 1;
}
else if(kicker_kick == 'Right'){
    selected_index += 2;
}

freq[selected_index] += 1;

//Update Goalie Decision Tracking:
    
selected_index = 0;
if(kicker_foot == 'Right'){
    selected_index += 3;
}
if(goalie_action == 'Middle'){
    selected_index += 1;
}
else if(goalie_action == 'Right'){
    selected_index += 2;
}

decisions[selected_index] += 1;

if(strategy_to_use.text == "Fictitious_Play"){

    //Update Goalie Perceived Risks:

    selected_index = 0;
    if(kicker_foot == 'Right'){
        selected_index += 3;
    }
    for(var i=0; i<=5; i++){
        perceived_risks[i] = 0;
    }
    perceived_risks[selected_index] = danger_goalie_left.toString().substring(0, 8);
    perceived_risks[selected_index + 1] = danger_goalie_middle.toString().substring(0, 8);
    perceived_risks[selected_index + 2] = danger_goalie_right.toString().substring(0, 8);
}
"""

select_bar = """
if (kicker_foot == 'Right'){
    selected_bar += 3;
}

if(kicker_kick == 'Middle'){
    selected_bar += 1;
}
else if(kicker_kick == 'Right'){
    selected_bar += 2;
}
"""

update_bar = """
var scored_bar_height = parseInt(scored_texts[selected_bar].text);
scored_bars[selected_bar].height = scored_bar_height;
scored_bars[selected_bar].y = scored_bar_height / 2;

var blockedl_bar_height = parseInt(blockedl_texts[selected_bar].text);
blockedl_bars[selected_bar].height = blockedl_bar_height;
blockedl_bars[selected_bar].y = scored_bar_height + blockedl_bar_height/2;

var blockedm_bar_height = parseInt(blockedm_texts[selected_bar].text);
blockedm_bars[selected_bar].height = blockedm_bar_height;
blockedm_bars[selected_bar].y = scored_bar_height + blockedl_bar_height + blockedm_bar_height/2;

var blockedr_bar_height = parseInt(blockedr_texts[selected_bar].text);
blockedr_bars[selected_bar].height = blockedr_bar_height;
blockedr_bars[selected_bar].y = scored_bar_height + blockedl_bar_height + blockedm_bar_height + blockedr_bar_height/2;
"""

resize_graph = """
var new_graph_height = 0;
for (var i = 0; i <= 5; i++){
    var possible_graph_height = (Math.round((parseInt(scored_texts[i].text)
                                             + parseInt(blockedl_texts[i].text)
                                             + parseInt(blockedm_texts[i].text)
                                             + parseInt(blockedr_texts[i].text))
                                            * 5/4));
    if(possible_graph_height > new_graph_height){
       new_graph_height = possible_graph_height;
    }
    game_stats_figure_1.y_range.end = new_graph_height;
}
"""

update_figure_1_source = """
if(parseInt(nround.text) >= parseInt(iterations_to_run.text)){
    const fig_1_data = game_stats_figure_1_source.data;
    
    for (var i = 0; i <= 5; i++){
        var scored_y_height = parseInt(scored_texts[i].text);
        var blockedl_height = parseInt(blockedl_texts[i].text) + scored_y_height;
        var blockedm_height = parseInt(blockedm_texts[i].text) + blockedl_height;
        var blockedr_height = parseInt(blockedr_texts[i].text) + blockedm_height;
        
        fig_1_data['scored_y'][i*3] = scored_y_height;
        fig_1_data['scored_y'][i*3 + 1] = scored_y_height;
        fig_1_data['scored_y'][i*3 + 2] = scored_y_height;
        
        fig_1_data['blockedl_y'][i*3] = blockedl_height;
        fig_1_data['blockedl_y'][i*3 + 1] = blockedl_height;
        fig_1_data['blockedl_y'][i*3 + 2] = blockedl_height;
        
        fig_1_data['blockedm_y'][i*3] = blockedm_height;
        fig_1_data['blockedm_y'][i*3 + 1] = blockedm_height;
        fig_1_data['blockedm_y'][i*3 + 2] = blockedm_height;
        
        fig_1_data['blockedr_y'][i*3] = blockedr_height;
        fig_1_data['blockedr_y'][i*3 + 1] = blockedr_height;
        fig_1_data['blockedr_y'][i*3 + 2] = blockedr_height;
    }
    
    game_stats_figure_1_source.change.emit();
}
"""

update_game_stats_figure_1 = """

var selected_bar = 0;

var scored_bars = [ll_scored_bar, lm_scored_bar, lr_scored_bar, 
                   rl_scored_bar, rm_scored_bar, rr_scored_bar];
var scored_texts = [ll_scored, lm_scored, lr_scored, 
                    rl_scored, rm_scored, rr_scored];
                    
var blockedl_bars = [ll_blocked_left_bar, lm_blocked_left_bar, lr_blocked_left_bar,
                     rl_blocked_left_bar, rm_blocked_left_bar, rr_blocked_left_bar];
var blockedl_texts = [ll_blocked_left, lm_blocked_left, lr_blocked_left, 
                      rl_blocked_left, rm_blocked_left, rr_blocked_left];

var blockedm_bars = [ll_blocked_middle_bar, lm_blocked_middle_bar, lr_blocked_middle_bar, 
                     rl_blocked_middle_bar, rm_blocked_middle_bar, rr_blocked_middle_bar];
var blockedm_texts = [ll_blocked_middle, lm_blocked_middle, lr_blocked_middle, 
                      rl_blocked_middle, rm_blocked_middle, rr_blocked_middle];

var blockedr_bars = [ll_blocked_right_bar, lm_blocked_right_bar, lr_blocked_right_bar, 
                     rl_blocked_right_bar, rm_blocked_right_bar, rr_blocked_right_bar];
var blockedr_texts = [ll_blocked_right, lm_blocked_right, lr_blocked_right, 
                      rl_blocked_right, rm_blocked_right, rr_blocked_right];

"""+select_bar+"""

if(goal == 1){

    var new_score = parseInt(scored_texts[selected_bar].text);
    new_score += 1;
    scored_texts[selected_bar].text = new_score.toString();

}
else{

    if(goalie_action == 'Left'){ 
        var new_blockedl = parseInt(blockedl_texts[selected_bar].text);
        new_blockedl += 1;
        blockedl_texts[selected_bar].text = new_blockedl.toString();
    }
    else if(goalie_action == 'Middle'){
        var new_blockedm = parseInt(blockedm_texts[selected_bar].text);
        new_blockedm += 1;
        blockedm_texts[selected_bar].text = new_blockedm.toString();
    }
    else{
        var new_blockedr = parseInt(blockedr_texts[selected_bar].text);
        new_blockedr += 1;
        blockedr_texts[selected_bar].text = new_blockedr.toString();
    }
}

""" + update_bar + resize_graph + update_figure_1_source

update_game_stats_figure_2 = """
const fig_2_data = game_stats_figure_2_source.data;
fig_2_data['ys'][parseInt(nround.text)] = parseInt(score.text);
game_stats_figure_2_source.change.emit();

if(parseInt(nround.text) >= parseInt(iterations_to_run.text)){
    //Resize Graph and Hitboxes:
    var fig_2_min_val = 0;
    var fig_2_max_val = 0;
    
    for(var i = 0; i <= parseInt(iterations_to_run.text); i++){
        if(fig_2_min_val > fig_2_data['ys'][i]){
            fig_2_min_val = fig_2_data['ys'][i];
        }
        if(fig_2_max_val < fig_2_data['ys'][i]){
            fig_2_max_val = fig_2_data['ys'][i];
        }
    }
    //Resize Hitboxes:
    var heights = [];
    var buffer = Math.round((Math.abs(fig_2_max_val) + Math.abs(fig_2_min_val)) * 1/8) + 1;
    if(Math.abs(fig_2_max_val) > Math.abs(fig_2_min_val)){
        for(var i = 0; i <= parseInt(iterations_to_run.text); i++){
            heights.push(Math.abs((fig_2_max_val + buffer) * 2));
        }
    }
    else{
        for(var i = 0; i <= parseInt(iterations_to_run.text); i++){
            heights.push(Math.abs((fig_2_min_val - buffer) * 2));
        }
    }
    fig_2_data['heights'] = heights;
    
    //Resize Graph:
    game_stats_figure_2.y_range.end = fig_2_max_val + buffer;
    game_stats_figure_2.y_range.start = fig_2_min_val - buffer;
    
    game_stats_figure_2_source.change.emit();
}
"""





automate_loop_iteration_main = (automate_loop_roll_kicker_action
                                + automate_loop_handle_goalie_decision
                                + automate_loop_handle_scoring
                                + automate_loop_animation
                                + automate_loop_update_fictitious_decision_tracking
                                + update_game_stats_figure_1
                                + update_game_stats_figure_2)

automate_loop_iteration=(automate_loop_iteration_var_instantiations
                         + automate_loop_iteration_main
                         + automate_loop_iteration_display)

b_automate_start_code = (automate_start_code_initial_gui_display
                         + automate_loop_iteration
                         )

b_start_automate_click = CustomJS(args = dict(b_start_automate = b_start_automate, b_auto_next = b_auto_next,
                                              LL_aim_slider = LL_aim_slider, LM_aim_slider = LM_aim_slider,
                                              LR_aim_slider = LR_aim_slider, RL_aim_slider = RL_aim_slider,
                                              RM_aim_slider = RM_aim_slider, RR_aim_slider = RR_aim_slider,
                                              iterations_slider = iterations_slider, strategy_dropdown = strategy_dropdown,
                                              automation_table = automation_table,
                                              automation_distribution_table = automation_distribution_table,
                                              ChancesColumnDataSource = automation_table_source,
                                              DistributionColumnDataSource = automation_distribution_table_source,
                                              txt = scr_text, strategy_to_use = strategy_to_use, nround = nround,
                                              iterations_to_run = iterations_to_run, game_figure = game_figure,
                                              game_stats_figure_1 = game_stats_figure_1, 
                                              game_stats_figure_2 = game_stats_figure_2, score = score,
                                              ball = ball, goalie_head = goalie_head, goalie_body = goalie_body,
                                              game_stats_figure_1_source = game_stats_figure_1_source,
                                              ll_scored_bar = ll_scored_bar, lm_scored_bar = lm_scored_bar,
                                              lr_scored_bar = lr_scored_bar, rl_scored_bar = rl_scored_bar,
                                              rm_scored_bar = rm_scored_bar, rr_scored_bar = rr_scored_bar,
                                              ll_scored = ll_scored, lm_scored = lm_scored,
                                              lr_scored = lr_scored, rl_scored = rl_scored,
                                              rm_scored = rm_scored, rr_scored = rr_scored,
                                              ll_blocked_left_bar = ll_blocked_left_bar,
                                              lm_blocked_left_bar = lm_blocked_left_bar,
                                              lr_blocked_left_bar = lr_blocked_left_bar,
                                              rl_blocked_left_bar = rl_blocked_left_bar,
                                              rm_blocked_left_bar = rm_blocked_left_bar,
                                              rr_blocked_left_bar = rr_blocked_left_bar,
                                              ll_blocked_left = ll_blocked_left, lm_blocked_left = lm_blocked_left,
                                              lr_blocked_left = lr_blocked_left, rl_blocked_left = rl_blocked_left,
                                              rm_blocked_left = rm_blocked_left, rr_blocked_left = rr_blocked_left,
                                              ll_blocked_middle_bar = ll_blocked_middle_bar,
                                              lm_blocked_middle_bar = lm_blocked_middle_bar,
                                              lr_blocked_middle_bar = lr_blocked_middle_bar,
                                              rl_blocked_middle_bar = rl_blocked_middle_bar,
                                              rm_blocked_middle_bar = rm_blocked_middle_bar,
                                              rr_blocked_middle_bar = rr_blocked_middle_bar,
                                              ll_blocked_middle = ll_blocked_middle, lm_blocked_middle = lm_blocked_middle,
                                              lr_blocked_middle = lr_blocked_middle, rl_blocked_middle = rl_blocked_middle,
                                              rm_blocked_middle = rm_blocked_middle, rr_blocked_middle = rr_blocked_middle,
                                              ll_blocked_right_bar = ll_blocked_right_bar,
                                              lm_blocked_right_bar = lm_blocked_right_bar,
                                              lr_blocked_right_bar = lr_blocked_right_bar,
                                              rl_blocked_right_bar = rl_blocked_right_bar,
                                              rm_blocked_right_bar = rm_blocked_right_bar,
                                              rr_blocked_right_bar = rr_blocked_right_bar,
                                              ll_blocked_right = ll_blocked_right, lm_blocked_right = lm_blocked_right,
                                              lr_blocked_right = lr_blocked_right, rl_blocked_right = rl_blocked_right,
                                              rm_blocked_right = rm_blocked_right, rr_blocked_right = rr_blocked_right,
                                              game_stats_figure_2_source = game_stats_figure_2_source),
                                  code = b_automate_start_code)

b_start_automate.js_on_click(b_start_automate_click)

In [20]:
# b_auto_next callback
b_auto_next_code = automate_loop_iteration

b_auto_next_click = CustomJS(args = dict(ChancesColumnDataSource = automation_table_source,
                                         DistributionColumnDataSource = automation_distribution_table_source,
                                         txt = scr_text, strategy_to_use = strategy_to_use, nround = nround,
                                         iterations_to_run = iterations_to_run, b_auto_next = b_auto_next,
                                         game_figure = game_figure, 
                                         automation_distribution_table = automation_distribution_table,
                                         game_stats_figure_1 = game_stats_figure_1, 
                                         game_stats_figure_2 = game_stats_figure_2, score = score, 
                                         ball = ball, goalie_head = goalie_head, goalie_body = goalie_body,
                                         game_stats_figure_1_source = game_stats_figure_1_source,
                                         ll_scored_bar = ll_scored_bar, lm_scored_bar = lm_scored_bar,
                                         lr_scored_bar = lr_scored_bar, rl_scored_bar = rl_scored_bar,
                                         rm_scored_bar = rm_scored_bar, rr_scored_bar = rr_scored_bar,
                                         ll_scored = ll_scored, lm_scored = lm_scored,
                                         lr_scored = lr_scored, rl_scored = rl_scored,
                                         rm_scored = rm_scored, rr_scored = rr_scored,
                                         ll_blocked_left_bar = ll_blocked_left_bar,
                                         lm_blocked_left_bar = lm_blocked_left_bar,
                                         lr_blocked_left_bar = lr_blocked_left_bar,
                                         rl_blocked_left_bar = rl_blocked_left_bar,
                                         rm_blocked_left_bar = rm_blocked_left_bar,
                                         rr_blocked_left_bar = rr_blocked_left_bar,
                                         ll_blocked_left = ll_blocked_left, lm_blocked_left = lm_blocked_left,
                                         lr_blocked_left = lr_blocked_left, rl_blocked_left = rl_blocked_left,
                                         rm_blocked_left = rm_blocked_left, rr_blocked_left = rr_blocked_left,
                                         ll_blocked_middle_bar = ll_blocked_middle_bar,
                                         lm_blocked_middle_bar = lm_blocked_middle_bar,
                                         lr_blocked_middle_bar = lr_blocked_middle_bar,
                                         rl_blocked_middle_bar = rl_blocked_middle_bar,
                                         rm_blocked_middle_bar = rm_blocked_middle_bar,
                                         rr_blocked_middle_bar = rr_blocked_middle_bar,
                                         ll_blocked_middle = ll_blocked_middle, lm_blocked_middle = lm_blocked_middle,
                                         lr_blocked_middle = lr_blocked_middle, rl_blocked_middle = rl_blocked_middle,
                                         rm_blocked_middle = rm_blocked_middle, rr_blocked_middle = rr_blocked_middle,
                                         ll_blocked_right_bar = ll_blocked_right_bar,
                                         lm_blocked_right_bar = lm_blocked_right_bar,
                                         lr_blocked_right_bar = lr_blocked_right_bar,
                                         rl_blocked_right_bar = rl_blocked_right_bar,
                                         rm_blocked_right_bar = rm_blocked_right_bar,
                                         rr_blocked_right_bar = rr_blocked_right_bar,
                                         ll_blocked_right = ll_blocked_right, lm_blocked_right = lm_blocked_right,
                                         lr_blocked_right = lr_blocked_right, rl_blocked_right = rl_blocked_right,
                                         rm_blocked_right = rm_blocked_right, rr_blocked_right = rr_blocked_right,
                                         game_stats_figure_2_source = game_stats_figure_2_source),
                             code = b_auto_next_code)

b_auto_next.js_on_click(b_auto_next_click)

In [21]:
aim_slider_callback_code = """
var data = ColumnDataSourceToChange.data;
var chances = data['chances'];

var total = 0;
total += LL_aim_slider.value;
total += LM_aim_slider.value;
total += LR_aim_slider.value;
total += RL_aim_slider.value;
total += RM_aim_slider.value;
total += RR_aim_slider.value;

chances[0] = LL_aim_slider.value / total;
chances[1] = LM_aim_slider.value / total;
chances[2] = LR_aim_slider.value / total;
chances[3] = RL_aim_slider.value / total;
chances[4] = RM_aim_slider.value / total;
chances[5] = RR_aim_slider.value / total;

ColumnDataSourceToChange.change.emit();
"""

aim_slider_customjs = CustomJS(args = dict(ColumnDataSourceToChange = automation_table_source,
                                       LL_aim_slider = LL_aim_slider, LM_aim_slider = LM_aim_slider, 
                                       LR_aim_slider = LR_aim_slider, RL_aim_slider = RL_aim_slider, 
                                       RM_aim_slider = RM_aim_slider, RR_aim_slider = RR_aim_slider), 
                           code = aim_slider_callback_code)

LL_aim_slider.js_on_change('value', aim_slider_customjs)
LM_aim_slider.js_on_change('value', aim_slider_customjs)
LR_aim_slider.js_on_change('value', aim_slider_customjs)
RL_aim_slider.js_on_change('value', aim_slider_customjs)
RM_aim_slider.js_on_change('value', aim_slider_customjs)
RR_aim_slider.js_on_change('value', aim_slider_customjs)

In [22]:
iterations_slider_code = """
var iterations = parseInt(iterations_to_run.text);
iterations = cb_obj.value;
iterations_to_run.text = iterations.toString();
game_stats_figure_1.y_range.end = iterations;

game_stats_figure_2.x_range.end = iterations;
game_stats_figure_2.y_range.start = -iterations;
game_stats_figure_2.y_range.end = iterations;

var xs = [];
var ys = [];
var heights = [];
var highlight_alphas = [];
for (var i = 0; i <= iterations; i++){
    xs.push(i);
    ys.push(0);
    heights.push(iterations * 2);
    highlight_alphas.push(0);
}
game_stats_figure_2_source.data['xs'] = xs;
game_stats_figure_2_source.data['ys'] = ys;
game_stats_figure_2_source.data['heights'] = heights;
game_stats_figure_2_source.data['highlight_alphas'] = highlight_alphas;

game_stats_figure_2_source.change.emit();
"""

iterations_slider_callback = CustomJS(args = dict(iterations_to_run = iterations_to_run,
                                                  game_stats_figure_1 = game_stats_figure_1,
                                                  game_stats_figure_2 = game_stats_figure_2,
                                                  game_stats_figure_2_source = game_stats_figure_2_source), 
                                      code = iterations_slider_code)

iterations_slider.js_on_change('value', iterations_slider_callback)

In [23]:
strategy_dropdown_code = """
strategy_dropdown.label = this.item;
strategy_to_use.text = this.item;
b_start_automate.visible = true;
"""

strategy_dropdown_callback = CustomJS(args = dict(strategy_dropdown = strategy_dropdown,
                                                  strategy_to_use = strategy_to_use,
                                                  b_start_automate = b_start_automate), 
                                      code = strategy_dropdown_code)

strategy_dropdown.js_on_event("menu_item_click", strategy_dropdown_callback)

### Format Layout and Show

In [24]:
buttons_bot = row(b_start, nround, b_fl, b_fr, b_shoot, bl, bm, br,
                  score, freq_left_foot, freq_right_foot, kicker_foot, 
                  kicker_kick, b_next, max_width = 600, sizing_mode = 'stretch_width')
automate_button = row(b_automate, iterations_slider,  b_auto_next, 
                      max_width = 400, sizing_mode = 'stretch_width')
strategy_dropdown_row = row(strategy_dropdown, max_width = 400, sizing_mode = 'stretch_width')
start_automate_row = row(b_start_automate, max_width = 400, sizing_mode = 'stretch_width')
automate_LL_aim = row(LL_aim_slider, max_width = 400, sizing_mode = 'stretch_width')
automate_LM_aim = row(LM_aim_slider, max_width = 400, sizing_mode = 'stretch_width')
automate_LR_aim = row(LR_aim_slider, max_width = 400, sizing_mode = 'stretch_width')
automate_RL_aim = row(RL_aim_slider, max_width = 400, sizing_mode = 'stretch_width')
automate_RM_aim = row(RM_aim_slider, max_width = 400, sizing_mode = 'stretch_width')
automate_RR_aim = row(RR_aim_slider, max_width = 400, sizing_mode = 'stretch_width')

game_stats_row_1 = row(game_stats_figure_1, game_stats_figure_2, 
                       max_width = 600, sizing_mode = "stretch_both")
gui_column1 = column(game_figure, game_stats_row_1, buttons_bot, 
                     max_width = 600, sizing_mode = 'stretch_width')
gui_column2 = column(automate_button, strategy_dropdown_row, start_automate_row,
                     automate_LL_aim, automate_LM_aim, automate_LR_aim,
                     automate_RL_aim, automate_RM_aim, automate_RR_aim,
                     automation_table, automation_distribution_table, 
                     min_width = 761, max_width = 761, sizing_mode = 'stretch_width')

gui_row = row(gui_column1, gui_column2, max_width = 1400, sizing_mode = 'stretch_width')

grid1 = gridplot([[gui_row]], plot_width = 1200, plot_height = 480)


show(grid1)

### Create HTML File

In [25]:
output_file("penalty_kick_game.html")
save(grid1)

'C:\\Users\\kylek\\Documents\\GitHub\\engri-1101-labs\\game_theory\\penalty_kick_game.html'